In [3]:
import requests
import random
from bs4 import BeautifulSoup
user_agents = [
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.1 Safari/605.1.15",
        "Mozilla/5.0 (Linux; Android 10; Pixel 3 XL Build/QP1A.190711.020) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Mobile Safari/537.36",
        "Mozilla/5.0 (iPhone; CPU iPhone OS 14_0 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1",
        "Mozilla/5.0 (Linux; Android 11; SM-G991B Build/RP1A.200720.012) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Mobile Safari/537.36"
    ]

headers = {
    "User-Agent": random.choice(user_agents),
    "Accept-Language": random.choice(["en-US,en;q=0.9", "fr-FR,fr;q=0.9", "es-ES,es;q=0.9"]),
    "Accept-Encoding": "gzip, deflate, br",
    "Connection": "keep-alive"
}

profile_name = 'Neas'

page_profile = requests.get(f"https://shikimori.one/{profile_name}", headers=headers)
page_watching = requests.get(f"https://shikimori.one/{profile_name}/list/anime", headers=headers)
soup_profile = BeautifulSoup(page_profile.text, 'html.parser')
soup_watching = BeautifulSoup(page_watching.text, 'html.parser')
with open('profile.html', 'w', encoding='utf-8') as file:
    file.write(str(soup_profile))
with open('watching.html', 'w', encoding='utf-8') as file:
    file.write(str(soup_watching))


# Что в списке у пользователя

In [4]:
watching_soup = soup_profile.find("div",class_='b-stats_bar anime')
for watch_type in ['Запланировано', 'Смотрю','Просмотрено','Отложено','Брошено']:
    selected_info = watching_soup.select(f"[data-type]:-soup-contains('{watch_type}') span")
    if selected_info:
        print(watch_type,selected_info[0].text)

Запланировано 799
Смотрю 260
Просмотрено 353
Отложено 6


# Парсинг истории

In [11]:
from tqdm import tqdm
url = "https://shikimori.one/Neas/history"
page_hist = requests.get(url, headers=headers)
soup_history = BeautifulSoup(page_hist.text, 'html.parser')
history_lines = soup_history.find_all('div',class_="b-user_history-line")
len(history_lines)

90

In [ ]:
import aiohttp
import asyncio
from bs4 import BeautifulSoup
import pandas as pd

parsed_history = []
async def fetch(session, url):
    async with session.get(url) as response:
        if response.status == 200:
            return await response.text()
        else:
            raise Exception(f"Ошибка при запросе: {response.status}")

async def parse_page(html):
    soup_history = BeautifulSoup(html, "html.parser")
    history_lines = soup_history.find_all('div',class_="b-user_history-line")
    
    if history_lines:
        return [[line.find('span',class_= "name-ru").text, line.find('span').get_text().split('\xa0')[1]] for line in history_lines if line.find('span',class_= "name-ru")]
    return None

async def main(base_url):
    global parsed_history
    async with aiohttp.ClientSession() as session:
        page_number = 1
        while True:
            url = f"{base_url}?page={page_number}"
            html = await fetch(session, url)
            result = await parse_page(html)
            if result:
                print(f"Страница {page_number}: {result}")
                parsed_history += result
            else:
                break
            page_number += 1

if __name__ == "__main__":
    base_url = "https://shikimori.one/Neas/history"  
    await main(base_url)


Страница 1: [['Жизнь ниндзя и убийцы', 'добавлено в список'], ['Дни проходят, а еда по-прежнему хороша', 'добавлено в список'], ['Карманы лета', 'добавлено в список'], ['Моя геройская академия: Финал', 'добавлено в список'], ['Моя геройская академия 7', 'добавлено в список'], ['Моя геройская академия: Вне закона', 'добавлено в список'], ['Негативный позитивный рыбак', 'добавлено в список'], ['Рубеж Шангри-Ла: Любитель игрошлака бросает вызов топ-игре 2', 'просмотрено'], ['Юно: Девушка, что воспевает любовь на грани этого мира', 'смотрю'], ['Плачущий призрак на пенсии', 'добавлено в список'], ['Рубеж Шангри-Ла: Любитель игрошлака бросает вызов топ-игре 2', 'просмотрено 11 эпизодов'], ['Рубеж Шангри-Ла: Любитель игрошлака бросает вызов топ-игре 2', 'смотрю'], ['Рубеж Шангри-Ла: Любитель игрошлака бросает вызов топ-игре', 'просмотрено'], ['Рубеж Шангри-Ла: Любитель игрошлака бросает вызов топ-игре', 'просмотрено 12 эпизодов'], ['Полное исследование', 'добавлено в список'], ['Плод эволюции

In [37]:
history_df = pd.DataFrame(parsed_history, columns=["Название", "Действие"])
temp = {}
for action_type in ['добавлено в список', 'просмотрено', 'смотрю']:
    temp[action_type] = history_df[history_df['Действие'] == action_type]['Название'].values
temp

{'добавлено в список': array(['Жизнь ниндзя и убийцы', 'Дни проходят, а еда по-прежнему хороша',
        'Карманы лета', ..., 'Охотник х Охотник (2011)', 'Врата Штейна',
        'Твоё имя'], shape=(1569,), dtype=object),
 'просмотрено': array(['Рубеж Шангри-Ла: Любитель игрошлака бросает вызов топ-игре 2',
        'Рубеж Шангри-Ла: Любитель игрошлака бросает вызов топ-игре',
        'Re:Zero. Перерыв с нуля 3',
        'Re:Zero. Жизнь с нуля в альтернативном мире 3',
        'Самый известный диктор создаёт самый великий в мире клан',
        'Реинкарнация безработного: История о приключениях в другом мире 2. Часть 2',
        'Реинкарнация безработного: История о приключениях в другом мире 2',
        'Реинкарнация безработного: История о приключениях в другом мире. Часть 2',
        'Дисквалифицирован по жизни', 'Перестану быть героем', 'Дандадан',
        'Невероятный новичок средних лет, прошедший тренировки на грани смерти',
        'Я парировал всё время, чтобы стать сильнейшим ав